In [1]:
from fludashboard import settings
from fludashboard.libs import episem
from numpy import ceil
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import fludashboard as flud
import pandas as pd
import sqlalchemy as sqla
import plotly.graph_objs as go
import colorlover as cl

In [2]:
init_notebook_mode(connected=True)
dsn = 'postgresql://%(USER)s:%(PASSWORD)s@%(HOST)s/%(NAME)s'
engine = sqla.create_engine(dsn % settings.DATABASE)

def ethiological_timeseries(scale_id=int, year=int, week=None, territory_id=None):
    """
    Generate timeseries for each ethiological agent and other relevant lab data
    
    :param scale_id: Incidence (0) or cases (1)
    :param year: epidemiological year
    :param week: epidemiological week or all (0)
    :param territory_id:
    """
    
    sql_param = {
        'epiweek': week,
        'epiyear': year,
        'territory_id': territory_id,
        'scale_id': scale_id
    }
    
    if week == None or week == 0:
        sql_param['epiweek'] = 53
    if territory_id == None:
        sql_param['territory_id'] = 0
    
    sql = '''
    SELECT
        notification.epiweek AS epiweek,
        notification.positive_cases AS "Testes positivos",
        notification.flu_a AS "Influenza A",
        notification.flu_b AS "Influenza B",
        notification.vsr AS "VSR",
        notification."ADNO" AS "Adenovirus",
        notification."PARA1" AS "Parainfluenza 1",
        notification."PARA2" AS "Parainfluenza 2",
        notification."PARA3" AS "Parainfluenza 3",
        notification.negative AS "Testes negativos",
        notification.not_tested AS "Casos sem teste laboratorial",
        notification.delayed AS "Casos aguardando resultado",
        notification.testing_ignored AS "Casos sem informação laboratorial",
        territory.name AS territory_name
    FROM
        (SELECT 
            epiweek,
            epiyear,
            dataset_id,
            scale_id,
            gender,
            positive_cases,
            flu_a,
            flu_b,
            vsr,
            "ADNO",
            "PARA1",
            "PARA2",
            "PARA3",
            negative,
            not_tested,
            delayed,
            testing_ignored,
            territory_id
        FROM
            clean_data_epiweek_weekly_incidence_w_situation
        WHERE
            epiyear=%(epiyear)s
            AND dataset_id=1
            AND territory_id=%(territory_id)s
            AND scale_id=%(scale_id)s
            AND gender='Total'
            AND epiweek <= %(epiweek)s
        ) as notification
        LEFT JOIN territory
            ON (notification.territory_id=territory.id)
        WHERE 1=1
    ORDER BY epiweek
    ''' % sql_param

    with engine.connect() as conn:
        return pd.read_sql(sql, conn)

In [3]:
df = ethiological_timeseries(scale_id=1, territory_id=43, year=2017)
df.head()

,epiweek,Testes positivos,Influenza A,Influenza B,VSR,Adenovirus,Parainfluenza 1,Parainfluenza 2,Parainfluenza 3,Testes negativos,Casos sem teste laboratorial,Casos aguardando resultado,Casos sem informação laboratorial,territory_name
0,1,3.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,18.0,1.0,0.0,4.0,Rio Grande do Sul
1,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,1.0,Rio Grande do Sul
2,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,0.0,0.0,1.0,Rio Grande do Sul
3,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,0.0,0.0,0.0,Rio Grande do Sul
4,5,4.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,15.0,1.0,0.0,2.0,Rio Grande do Sul


In [4]:
def ethio_ts(df=pd.DataFrame, scale_id=int, year=int):
    cols = [
            'Testes positivos',
            'Influenza A',
            'Influenza B',
            'VSR',
            'Adenovirus',
            'Parainfluenza 1',
            'Parainfluenza 2',
            'Parainfluenza 3'
            ]
    trace = []
    trace.append(
            go.Scatter(
            x = df['epiweek'],
            y = df[cols[0]],
            name=cols[0],
            mode='lines',
            )
        )
    
    if scale_id == 2:
        ytitle = 'Casos'
    else:
        ytitle = 'Incidência'

    for i,col in enumerate(cols[1:]):
        trace.append(
            go.Scatter(
            x = df['epiweek'],
            y = df[col],
            name=ytitle,
            mode='lines',
            showlegend=False
            )
        )
    
    nrows = len(cols)
    fig = tools.make_subplots(
        rows=nrows,
        cols=1,
        print_grid=False,
        subplot_titles=('Situação dos exames',
            'Influenza A',
            'Influenza B',
            'VSR',
            'Adenovirus',
            'Parainfluenza 1',
            'Parainfluenza 2',
            'Parainfluenza 3'))
    
    for i in range(1,(nrows+1)):
        fig.append_trace(trace[i-1], i, 1)

    # Add extra lines to first subplot:
    extra_cols = ['Testes negativos',
                  'Casos sem teste laboratorial',
                  'Casos aguardando resultado',
                  'Casos sem informação laboratorial']
    for i,col in enumerate(extra_cols):
        fig.append_trace(
            go.Scatter(
            x = df['epiweek'],
            y = df[col],
            name=col,
            mode='lines',
            line=dict(color=cl.scales['12']['qual']['Paired'][-i])
            ), 1, 1)
    
    # X-axis title and range:
    lastepiweek = int(episem.episem(episem.lastepiday(year), out='W'))
    extra_cols.extend(['Testes positivos'])
    ymax = [
        max(5*ceil(df[extra_cols].max().max()/5), 5),
        max(5*ceil(df[cols[1:]].max().max()/5), 5)
    ]
    for i in range(1,(nrows+1)):
        xaxis = 'xaxis%s' % i
        yaxis = 'yaxis%s' % i
        fig['layout'][xaxis].update(range=[1, lastepiweek])
        fig['layout'][yaxis].update(range=[0, ymax[min(1,i-1)]], rangemode='nonnegative')
    
    fig['layout']['xaxis%s' % nrows].update(title='Semana epidemiológica', zeroline=True, showline=True)
    
    i = int(nrows/2)
    fig['layout']['yaxis%s' % i].update(title=ytitle)
    
    territory_lbl = df['territory_name'].unique()[0]
    fig['layout'].update(height=1600,
                         width=800,
                         title='Exames laboratoriais - %s' % territory_lbl
                        )
    iplot(fig)

In [5]:
ethio_ts(df, scale_id = 1, year=2017)

In [6]:
df = ethiological_timeseries(scale_id=1, territory_id=43, year=2017, week=15)
ethio_ts(df, scale_id = 1, year=2017)